In [1]:
import pandas as pd
from data_columns import *

In [12]:
# Define a function for concatenation
def concatenate_values(series):
    return ' '.join(series.dropna().astype(str))

# Split the string into a list of words, remove duplicates, and maintain original order
def remove_duplicates_from_string(s):
    return list(sorted(set(s.split()), key=s.split().index))

#Define a function to remove duplicates and join codes
def remove_duplicates_concatenate(df, columns):
    df = df.groupby('source_url', as_index=False).agg({
    col: concatenate_values for col in columns
    })

    for col in columns:
        df[col] = df[col].apply(remove_duplicates_from_string)

    return df

def map_codes_to_description(codes_list, code_dictionary):
    descriptions = []
    for code in codes_list:
        if len(code) == 3:  # Ensure the code is 3 characters
            descriptions.append(code_dictionary.get(code, ''))
        elif len(code) % 3 == 0:
            # Code length is a multiple of 3
            chunks = [code[i:i+3] for i in range(0, len(code), 3)]
            chunk_descriptions = [code_dictionary.get(chunk, '') for chunk in chunks]
            descriptions.append(' '.join(chunk_descriptions))
        else:
            continue
    return ' '.join(descriptions)

def remove_duplicates_description(text):
    words = text.split()
    seen = set()
    unique_words = []
    
    for word in words:
        if word not in seen:
            seen.add(word)
            unique_words.append(word)
    
    return ' '.join(unique_words)

In [3]:
codes_letters = ['actor1_code', 'actor1_country_code', 'actor1_known_group_code', 'actor1_type1_code', 
         'actor2_code', 'actor2_country_code', 'actor2_known_group_code', 'actor2_type1_code']

codes_nums = ['event_code']

other_important = ['actor1_name', 'actor2_name', 'actor1_geo_full_name','actor2_geo_fullname']

actors = pd.read_csv("Entity_Codes/actor_codes.csv")
countries = pd.read_csv("Entity_Codes/country_codes.csv")
codes = pd.concat([actors, countries], ignore_index=True)
code_dictionary = dict(zip(codes['Code'], codes['Description']))

In [5]:
df = pd.read_csv('./Data_Storage/GDELT Event Files/20240813.export.CSV', sep = '\t', header = None)
df.columns = column_names

df = remove_duplicates_concatenate(df, codes_letters)

/var/folders/_s/n53dl30x6x90zrn97fjh8_6m0000gn/T/ipykernel_82111/3249109898.py:1: DtypeWarning: Columns (14,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./Data_Storage/GDELT Event Files/20240813.export.CSV', sep = '\t', header = None)


In [13]:
# Initialize the descriptions column
df['description'] = ''

# Iterate over each column and append descriptions
for column in codes_letters:
    df['description'] += df[column].apply(lambda x: map_codes_to_description(x, code_dictionary)) + ' '

df['description'] = df['description'].apply(remove_duplicates_description)

df[['source_url', 'description']]
    

,source_url,description
0,http://016.jose947.com/,United States of America Business
1,http://0kcw.jose947.com/,Agriculture
2,http://africa.chinadaily.com.cn/a/202408/13/WS...,"China Government, political parties United Kin..."
3,http://africa.chinadaily.com.cn/a/202408/13/WS...,China
4,http://agenda.ge/en/news/2024/40172,Azerbaijan Hungary Romania United States of Am...
...,...,...
28634,https://zeenews.india.com/world/who-is-faiz-ha...,Military Pakistan
28635,https://zenopa.com/news/fda-approves-ascendis-...,"Government, political parties"
28636,https://znsbahamas.com/pm-davis-keen-to-see-sm...,"Bahamas Government, political parties ""Health"""
28637,https://znsbahamas.com/residents-unite-against...,Civilians Bahamas
